In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
from tqdm import tqdm
import pickle

/usr/local/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
file_path = 'MOTHER_TABLE.csv'
df = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)
df

<ipython-input-2-0d37fca4f18a>:2: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)


,file_name,n,age,build,complexion,conscription,wage,remarks,f_n,zot_title,...,start_loc,last_loc,ship_code,last_name,first_name,alternate_last_name,alternate_first_name,parents,gender,families_groups
0,1722 - Rôle du Solide_2028,1.0,NaN,NaN,NaN,NaN,75,a fait la campagne,NaN,Rôle du Solide (1722),...,Lorient,Lorient,SOL_B,DE LA GLOIRAIS MANCEL,NaN,MANCEL,NaN,NaN,m,NaN
1,1722 - Rôle du Solide_2028,6.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,Lorient,Lorient,SOL_B,LEGER,Charles,NaN,NaN,NaN,m,NaN
2,1722 - Rôle du Solide_2028,7.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,Lorient,Lorient,SOL_B,BOUEZAR,Étienne,NaN,NaN,NaN,m,NaN
3,1722 - Rôle du Solide_2028,8.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,Lorient,Lorient,SOL_B,BASSET,Guillaume,NaN,NaN,NaN,m,NaN
4,1722 - Rôle du Solide_2028,3.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,Lorient,Lorient,SOL_B,BOUEDELAN,Pierre,NaN,NaN,NaN,m,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215659,ROLE2P44-II.9_2047,807.0,NaN,NaN,NaN,NaN,6,complément à l'île de France le 08/08/1770 \n ...,NaN,Rôle du Mars (1770 -,...,Lorient,Lorient,MRS_E,NaN,Dosmamothe,NaN,NaN,NaN,m,NaN
215660,ROLE2P44-II.9_2047,808.0,NaN,NaN,NaN,NaN,6,complément à l'île de France le 08/08/1770 \n ...,NaN,Rôle du Mars (1770 -,...,Lorient,Lorient,MRS_E,PIQUENINE,Goulabe,NaN,NaN,NaN,m,NaN
215661,ROLE2P51-34_1847,101.0,NaN,NaN,NaN,NaN,sans solde,remplacement à l'île de France le 17/12/1785 \...,NaN,Rôle de la Bretagne (1785),...,Bordeaux,Lorient,BRG_A,BURELLE,Guillaume,NaN,NaN,NaN,m,NaN
215662,ROLE2P39-II.9_1645,286.0,NaN,NaN,NaN,NaN,7.10,remplacement à Wampou le 02/12/1763 \n débarqu...,NaN,Rôle du Beaumont (1763 - 1764),...,Lorient,Lorient,BMT_B,JOSSE,Jacques,NaN,NaN,Pierre,m,NaN


In [3]:
def clean_text(text):
    if pd.isna(text):
        return ""
    return text.strip().lower()
df['last_name_clean'] = df['last_name'].apply(clean_text)
df['first_name_clean'] = df['first_name'].apply(clean_text)
df['parents_clean'] = df['parents'].apply(clean_text)
df['origin_clean'] = df['origin'].apply(clean_text)
df['remarks_clean'] = df['remarks'].apply(clean_text)
df['date_of_birth'] = pd.to_numeric(df['date_of_birth'], errors='coerce')
df['date_voy_begin'] = pd.to_datetime(df['date_voy_begin'], format='%d/%m/%Y')
df['date_voy_end'] = pd.to_datetime(df['date_voy_end'], format='%d/%m/%Y')
def filter_by_remarks(df, terms):
    pattern = '|'.join(terms)
    return df[df['remarks_clean'].str.contains(pattern, na=False)]
terms = ["provient", "provenant", "vient", "passé à bord", "rembarqué à bord",
         "passé sur", "repassé à bord", "renversé sur", "renversement"]

filtered_df = filter_by_remarks(df, terms)
def extract_ship_name(remark, terms):
    for term in terms:
        if term in remark:
            parts = remark.split(term)
            if len(parts) > 1:
                return parts[-1].strip().split(' ')[0]
    return np.nan
filtered_df['ship_name'] = filtered_df['remarks_clean'].apply(lambda x: extract_ship_name(x, terms))
def calculate_certainty(row, match_row):
    score = 0
    score += fuzz.ratio(row['last_name_clean'], match_row['last_name_clean']) / 100
    score += fuzz.ratio(row['first_name_clean'], match_row['first_name_clean']) / 100
    score += fuzz.ratio(row['parents_clean'], match_row['parents_clean']) / 100
    score += 1 if row['origin_clean'] == match_row['origin_clean'] else 0
    score += 1 if abs(row['date_of_birth'] - match_row['date_of_birth']) <= 5 else 0
    score += 1 if safe_compare(row['function_name'], match_row['function_name']) else 0
    return score

def safe_compare(val1, val2):
    if pd.isna(val1) or pd.isna(val2):
        return False
    return val1 == val2

def voyage_date_criteria(row, match_row):
    if pd.isna(row['date_voy_begin']) or pd.isna(match_row['date_voy_begin']):
        return True
    if abs((row['date_voy_begin'] - match_row['date_voy_begin']).days) / 365.25 > 20:
        return False
    if (row['date_voy_begin'] <= match_row['date_voy_end'] and
        row['date_voy_end'] >= match_row['date_voy_begin']):
        return False
    return True

<ipython-input-3-12b2037b54ef>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ship_name'] = filtered_df['remarks_clean'].apply(lambda x: extract_ship_name(x, terms))


In [4]:
def optimized_rule_3(df):
    matches = []
    print("Running Rule 3")

    # Precompute masks for constant conditions
    last_name_groups = df.groupby('last_name_clean')
    first_name_groups = df.groupby('first_name_clean')
    parents_groups = df.groupby('origin_clean')

    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Combine groups for filtering
        potential_matches = df[
            (df['last_name_clean'] == row['last_name_clean']) &
            (df['first_name_clean'] == row['first_name_clean']) &
            (df['origin_clean'] == row['origin_clean'])
        ].copy()

        # Filter based on voyage date criteria
        potential_matches = potential_matches[
            potential_matches.apply(lambda match_row: voyage_date_criteria(row, match_row), axis=1)
        ]

        for _, match_row in potential_matches.iterrows():
            certainty_score = calculate_certainty(row, match_row)
            matches.append((i, match_row.name, certainty_score))

    return matches

In [5]:
rule_3_matches = optimized_rule_3(df)
print(f"Rule 3 matches found: {len(rule_3_matches)}")

Running Rule 3


100%|██████████| 215664/215664 [2:49:31<00:00, 21.20it/s]  


Rule 3 matches found: 482492


In [6]:
with open('rule_3_matches.pickle', 'wb') as f:
    pickle.dump(rule_3_matches, f)